In [3]:
from jesse import helpers, research

warmup_candles, trading_candles = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "3m",
    helpers.date_to_timestamp("2020-01-01"),
    helpers.date_to_timestamp("2025-01-31"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

In [5]:
from offline.labeling.labeling import (
    TripleBarrierLabeler,
    expand_labels,
    return_of_label,
)

for i in [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02]:
    num_minutes = 16
    labeler = TripleBarrierLabeler(
        trading_candles, num_minutes=num_minutes, verbose=False
    )
    target_ret = i
    side_labels = labeler.side_labels(target_ret=target_ret)
    meta_labels = labeler.meta_labels(side_labels, target_ret=target_ret)
    candle_df = expand_labels(meta_labels, trading_candles, fill=0)
    ret = return_of_label(candle_df)
    label_count = candle_df["side"].value_counts().sort_index().to_numpy()
    print(f"{target_ret = } -> {ret = }, {label_count = }")

target_ret = 0.001 -> ret = 25.101523431478295, label_count = array([ 23162, 845598,  22600])
target_ret = 0.002 -> ret = 24.694560978439043, label_count = array([ 21658, 848820,  20882])
target_ret = 0.003 -> ret = 23.09779046896597, label_count = array([ 18474, 855407,  17479])


KeyboardInterrupt: 

In [6]:
from offline.labeling.labeling import (
    TripleBarrierLabeler,
    expand_labels,
    return_of_label,
)

for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
    num_minutes = 16
    target_ret = 0.002
    pt_sl_ratio = i
    labeler = TripleBarrierLabeler(
        trading_candles, num_minutes=num_minutes, verbose=False
    )
    side_labels = labeler.side_labels(
        pt=pt_sl_ratio, sl=pt_sl_ratio, target_ret=target_ret
    )
    meta_labels = labeler.meta_labels(
        side_labels, pt=pt_sl_ratio, sl=pt_sl_ratio, target_ret=target_ret
    )
    candle_df = expand_labels(meta_labels, trading_candles, fill=0)
    ret = return_of_label(candle_df)
    label_count = candle_df["side"].value_counts().sort_index().to_numpy()
    print(f"{pt_sl_ratio = } -> {ret = }, {label_count = }")

pt_sl_ratio = 0.1 -> ret = 43.85897840766976, label_count = array([312344, 264536, 314480])
pt_sl_ratio = 0.2 -> ret = 66.85736013416223, label_count = array([242143, 405089, 244128])
pt_sl_ratio = 0.3 -> ret = 74.61267795957251, label_count = array([173273, 543040, 175047])
pt_sl_ratio = 0.4 -> ret = 69.90182614236812, label_count = array([122639, 645247, 123474])
pt_sl_ratio = 0.5 -> ret = 60.76260672292184, label_count = array([ 87403, 715875,  88082])


KeyboardInterrupt: 

In [7]:
from offline.labeling.labeling import (
    TripleBarrierLabeler,
    expand_labels,
    return_of_label,
)

for i in [
    0.25,
    0.26,
    0.27,
    0.28,
    0.29,
    0.3,
    0.31,
    0.32,
    0.33,
    0.34,
    0.35,
]:
    num_minutes = 16
    target_ret = 0.002
    pt_sl_ratio = i
    labeler = TripleBarrierLabeler(
        trading_candles, num_minutes=num_minutes, verbose=False
    )
    side_labels = labeler.side_labels(
        pt=pt_sl_ratio, sl=pt_sl_ratio, target_ret=target_ret
    )
    meta_labels = labeler.meta_labels(
        side_labels, pt=pt_sl_ratio, sl=pt_sl_ratio, target_ret=target_ret
    )
    candle_df = expand_labels(meta_labels, trading_candles, fill=0)
    ret = return_of_label(candle_df)
    label_count = candle_df["side"].value_counts().sort_index().to_numpy()
    print(f"{pt_sl_ratio = } -> {ret = }, {label_count = }")

pt_sl_ratio = 0.25 -> ret = 72.8912460305981, label_count = array([205946, 477950, 207464])
pt_sl_ratio = 0.26 -> ret = 73.43068803701152, label_count = array([199055, 491552, 200753])
pt_sl_ratio = 0.27 -> ret = 73.9695223446967, label_count = array([192395, 504844, 194121])
pt_sl_ratio = 0.28 -> ret = 74.41269389820111, label_count = array([185733, 517985, 187642])
pt_sl_ratio = 0.29 -> ret = 74.40377041931781, label_count = array([179390, 530737, 181233])
pt_sl_ratio = 0.3 -> ret = 74.61267795957251, label_count = array([173273, 543040, 175047])
pt_sl_ratio = 0.31 -> ret = 74.39319656402596, label_count = array([167385, 554863, 169112])
pt_sl_ratio = 0.32 -> ret = 74.21904969867926, label_count = array([161736, 566322, 163302])


KeyboardInterrupt: 

In [4]:
import optuna
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from offline.labeling.labeling import (
    TripleBarrierLabeler,
    expand_labels,
)

feature_df = pd.read_parquet("data/features_d002.parquet")


def objective(trial):
    num_minutes = 16
    target_ret = trial.suggest_float("target_ret", 0.001, 0.008)
    pt = trial.suggest_float("pt", 0.1, 3)
    sl = trial.suggest_float("sl", 0.1, 3)
    labeler = TripleBarrierLabeler(
        trading_candles, num_minutes=num_minutes, verbose=False
    )
    side_labels = labeler.side_labels(pt=pt, sl=sl, target_ret=target_ret)
    meta_labels = labeler.meta_labels(side_labels, pt=pt, sl=sl, target_ret=target_ret)
    candle_df = expand_labels(meta_labels, trading_candles, fill=0)
    label_series = candle_df["bin"].astype(int).reset_index(drop=True)
    train_test_split_point = int(len(label_series) * 0.8)
    mask = candle_df["ret"].notna().values

    gap = 300
    train_label_series = label_series.iloc[gap:train_test_split_point]
    train_feature_df = feature_df.iloc[gap:train_test_split_point]
    train_mask = mask[gap:train_test_split_point]
    test_label_series = label_series.iloc[train_test_split_point + gap :]
    test_feature_df = feature_df.iloc[train_test_split_point + gap :]
    test_mask = mask[train_test_split_point + gap :]

    train_feature_df = train_feature_df[train_mask]
    train_label_series = train_label_series[train_mask]

    test_feature_df = test_feature_df[test_mask]
    test_label_series = test_label_series[test_mask]

    # if test_label_series is all 0, return 0
    if (test_label_series == 0).all():
        return 0

    clf = LGBMClassifier(
        num_leaves=200,
        n_estimators=500,
        max_depth=30,
        n_jobs=-1,
        class_weight={
            0: 1,
            1: int((train_label_series == 0).sum() / (train_label_series == 1).sum()),
        },
        verbose=-1,
    )

    clf.fit(train_feature_df, train_label_series)
    pred_proba = clf.predict_proba(test_feature_df)[:, 1]
    auc = roc_auc_score(test_label_series, pred_proba, average="weighted")
    precision = precision_score(test_label_series, (pred_proba > 0.5).astype(int))
    recall = recall_score(test_label_series, (pred_proba > 0.5).astype(int))
    f1 = f1_score(test_label_series, (pred_proba > 0.5).astype(int))
    print(f"{auc = }, {precision = }, {recall = }, {f1 = }")
    return f1


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2025-02-15 17:59:35,500] A new study created in memory with name: no-name-2a0b8a9f-64c1-4409-8053-f58801db52f2
[I 2025-02-15 18:02:41,035] Trial 0 finished with value: 0.07368421052631578 and parameters: {'target_ret': 0.007231525532107812, 'pt': 1.389676052424955, 'sl': 2.724999179653703}. Best is trial 0 with value: 0.07368421052631578.


auc = 0.8108842377030104, precision = 1.0, recall = 0.03825136612021858, f1 = 0.07368421052631578


/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-02-15 18:05:50,357] Trial 1 finished with value: 0.0 and parameters: {'target_ret': 0.0058737829794734495, 'pt': 2.4464594075046384, 'sl': 2.673247363223961}. Best is trial 0 with value: 0.07368421052631578.


auc = 0.8475527752136669, precision = 0.0, recall = 0.0, f1 = 0.0


/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2025-02-15 18:08:59,890] Trial 2 finished with value: 0.0 and parameters: {'target_ret': 0.005861210895460753, 'pt': 2.429125275324641, 'sl': 2.135505799782554}. Best is trial 0 with value: 0.07368421052631578.


...

auc = 0.7834354936692788, precision = 0.517874090477697, recall = 0.3045298111803553, f1 = 0.3835295495812101


[I 2025-02-16 01:52:35,006] Trial 98 finished with value: 0.04381443298969072 and parameters: {'target_ret': 0.0033293099175703547, 'pt': 1.5606072380745433, 'sl': 2.6713635444888655}. Best is trial 60 with value: 0.6885594069324784.


auc = 0.7817500198148951, precision = 0.5862068965517241, recall = 0.022757697456492636, f1 = 0.04381443298969072


[I 2025-02-16 01:57:31,569] Trial 99 finished with value: 0.5678399971655831 and parameters: {'target_ret': 0.0030663426145002234, 'pt': 0.2519553881324071, 'sl': 2.589038287993802}. Best is trial 60 with value: 0.6885594069324784.


auc = 0.7841337778277176, precision = 0.603335341063093, recall = 0.5362891082482851, f1 = 0.5678399971655831


In [8]:
from offline.labeling.labeling import (
    TripleBarrierLabeler,
    expand_labels,
    return_of_label,
)

num_minutes = 16
target_ret = 0.002
pt_sl_ratio = 0.3
labeler = TripleBarrierLabeler(trading_candles, num_minutes=num_minutes, verbose=True)
side_labels = labeler.side_labels(pt=pt_sl_ratio, sl=pt_sl_ratio, target_ret=target_ret)
meta_labels = labeler.meta_labels(
    side_labels, pt=pt_sl_ratio, sl=pt_sl_ratio, target_ret=target_ret
)
candle_df = expand_labels(meta_labels, trading_candles, fill=0)
ret = return_of_label(candle_df)
label_count = candle_df["side"].value_counts().sort_index().to_numpy()
print(f"{target_ret = } -> {ret = }, {label_count = }")

2025-02-17 23:51:52.721627 100.0% _apply_pt_sl_on_t1 done after 0.05 minutes. Remaining 0.0 minutes..
2025-02-17 23:53:11.817526 100.0% _apply_pt_sl_on_t1 done after 0.05 minutes. Remaining 0.0 minutes..


target_ret = 0.002 -> ret = 74.61267795957251, label_count = array([173273, 543040, 175047])


In [9]:
trading_candles.shape

(891360, 6)

In [10]:
print(candle_df.shape)
candle_df.head()

(891360, 5)


,close,ret,trgt,bin,side
2020-01-01 00:00:00+00:00,7177.93,NaN,NaN,0.0,0.0
2020-01-01 00:03:00+00:00,7175.61,NaN,NaN,0.0,0.0
2020-01-01 00:06:00+00:00,7182.06,NaN,NaN,0.0,0.0
2020-01-01 00:09:00+00:00,7183.07,NaN,NaN,0.0,0.0
2020-01-01 00:12:00+00:00,7176.26,NaN,NaN,0.0,0.0


In [11]:
candle_df["trgt"].isna().sum()

223299

In [12]:
candle_df.to_parquet("data/label_d002.parquet")